### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
# подключю пандас, присваиваю псевдоним для краткости

import pandas as pd

In [2]:
# чтаю данные из файла

df_movies = pd.read_csv('ratings.csv')
df_movies.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [3]:
# пишу функцию которая будет принимать на вход ЗНАЧЕНИЯ ->
# Значения, по тому что далее в заново формируемом мною df_movies['rating_level'] перед .apply  я обращаюсь к типу series

def rating_level (row):
    """ 
    Присваивает строковый уровень рейтинга, в зависимости от оценки рейтина столбца 'rating' 
    """
    result = ''
    if row <= 2:
        result = 'низкий рейтинг'
    elif row <= 4.5:
        result = 'средний рейтинг'
    elif row <= 5:
        result = 'высокий рейтинг'
    return result

In [4]:
# завожу новый столбец df_movies['rating_level'], 
# результат которого формируется методом apply, примененному к столбцу (тип series), 
# с помощью функции rating_level через метод apply

df_movies['rating_level'] = df_movies['rating'].apply(rating_level)
df_movies.head()

,userId,movieId,rating,timestamp,rating_level
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


In [5]:
# пишу версию_1 функции rating_level
# отличие от предыдущей в том, что на вход подается датафрей, а не число, по этому в ней есть уточнение к какому столбцу применять проверку
# (более универсальный вариант, по тому что используя функцию в aply без указания столбца, я осталяю возможность дописать в самой функции расчет по другим столбца. Это не оптимально с точки зрения затраты ресурсов, но в аналитике в данным случае это не присоедуется)
def rating_level_1(row):
    result = ''
    if row['rating'] <= 2:
        result = 'низкий рейтинг'
    elif row['rating'] <= 4.5:
        result = 'средний рейтинг'
    elif row['rating'] <= 5:
        result = 'высокий рейтинг'
    return result

In [6]:
df_movies['rating_level'] = df_movies.apply(rating_level_1, axis=1)
df_movies.head()

,userId,movieId,rating,timestamp,rating_level
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


### Задание 2
Используем файл keywords.csv.

- Необходимо написать гео-классификатор, 
- Который каждой строке сможет выставить географическую принадлежность определенному региону. 
- Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона.  
- Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

---

*Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:*

geo_data = {  

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']  

}

---
Результат классификации запишите в отдельный столбец region.

In [13]:
# правила распределния записаны в словаре:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'],
}
geo_data

{'Центр': ['москва', 'тула', 'ярославль'],
 'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

In [15]:
# читаю данные из файла

df_keywords = pd.read_csv('keywords.csv')
df_keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [16]:
# пишу функция которая классифицирует запросы по регионам
# Цикла for in перебирае справочник регионов
# Каждую строку справочника перебирает вложенный цикл, и делает проверку вхождения города региона в каждый запрос
# При нахождении города функция возвращает регион, который находится в ключе словаря справочника регионов
# Если в ключе не содержиться ни одного города из регионов, функция возвращает undefined

def geo_class(row):
    """
    Классифицирует запросы по регионам
    Классификация происходит в зависимости от содержания названия города региона в запрос
    """
    
    for region, city_list in geo_data.items():
        for city in city_list:
            if city in row['keyword']:
                return region
    return 'undefined'

In [17]:
# применяю функцию 'geo_class' через метод apply ко всему датафрейму, 
# а результат функции прописываю в заведенный столбец df_keywords['region']

df_keywords['region'] = df_keywords.apply(geo_class, axis=1)
df_keywords.head(2)

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined


In [18]:
# проверяю, прописываются ли регионы, и сколько таких запросов

df_keywords['region'].value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64